# Interactive Data Analysis Exercise

This is an exercise that was created to demonstrate use of [*Quibbler*](https://github.com/Technion-Kishony-lab/quibbler) package in qualitative data exploration and analysis.
<br>
The data used in this demonstration is the ICD-9-CM codes embeddings that were created by Y. Choi et al. in their paper [Learning Low-Dimensional Representations of Medical Concepts](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5001761). The embeddings were preprocessed to filter any non-relevant codes for this excercise.

<div align="center">
    <img src="https://raw.githubusercontent.com/Technion-Kishony-lab/quibbler/master/pyquibbler-documentations/docs/images/quibicon.gif" alt="quib icon" width="300">
</div>

The data exploration stage is an important step in the data science or analyist process, as it allows us to gain insights and understand patterns in our data. Good visualizations are crucial in this stage as they can provide intuitions and reveal patterns that may be hidden otherwise. In this exercise, we will be focusing on exploring the embedding space of our data to find ICD9 codes that are interesting to further investigate. We will also try to gain a deeper understanding of the data, labels, and biases through this process. To guide our exploration, we will be looking for things like:
1) Codes that form clusters
2) Codes that are anomalies within their surroundings in the embedding space
3) Codes that are distant from their close relatives codes (e.g. codes within the same chapter or group but distant in the embedding space)

<div align="center">
  <img src="https://i.ibb.co/JcmWT9C/scatter-examples-marked.png" />
</div>

In this exercise, we will utilize *Quibbler* to construct an interactive application that displays downstream information as we search for intriguing codes. This serves as a preview of the capabilities of the application we will be building in the subsequent code in this notebook.

<div align="center">
  <img src="https://im3.ezgif.com/tmp/ezgif-3-398dfcc8fa.gif" />
</div>

This code in this notebook is just an example. **You are encourged to a play with the app, add features you find relevant and change it to behave as you wish.**

In [ ]:
# if you dont have matplotlib, numpy or pandas installed on your machine uncomment this lines and pip install them:
# %pip install matplotlib
# %pip install numpy
# %pip install pandas

# and of course install quibbler!
%pip install pyquibbler

# you can also install the jupyter lab extenstion which can help you save and load quibs states
%pip install pyquibbler_labextension

In [ ]:
from pyquibbler import iquib, initialize_quibbler, q, quiby, set_project_directory, save_quibs, load_quibs
initialize_quibbler()
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import matplotlib.patches as patches
import numpy as np
import pandas as pd
# define matplotlib variable according to your platform
%matplotlib tk

## Load Data

The `df` columns:
- **code** - ICD-9-CM (ICD9 code in short) code
- **description** - Text descripion of the code
- **frequency** - The frequency of code occurance in the cohort
- **ICD9_chapter** - The association of ICD9 code to chapter, as determined by the ICD9 committee
- **CDC_group** - The association of ICD9 code to group, as determined by the CDC
- **factorized_ICD9_chapter** - Encoding of ICD9_chapter as integer, useful for filtering and plotting
- **TSNE_1** - First axis of TSNE
- **TSNE_2** - Second axis of TSNE

`chapters_abb` is a dictionary that maps ICD9 chapter abbreviation to full name.

In [ ]:
df = pd.read_pickle('data/ICD_df.pkl')
chapters_abb = pd.read_pickle('data/chapters_abb.pkl')

## Define `quiby` functions

Define any functions that you will need that are going to deal with quibs and are not already modified to work directly with quib arguments. If you don't know if a function is modified you can use `is_quiby()` [function](https://quibbler.readthedocs.io/en/latest/functions/pyquibbler.is_quiby.html#pyquibbler.is_quiby).

In [ ]:
@quiby
def filter_by_frequency(frequencies, frequency_threshold):
    return np.where(frequencies > frequency_threshold)

@quiby
def filter_text_by_frequency(bins_height, cluster_threshold, data):
    below_cluster_threshold = np.where((bins_height < cluster_threshold) & (bins_height > 1e-6))
    rows_with_relevant_clusters = data[np.where(np.isin(data[:, 5], below_cluster_threshold)), 1].flatten()
    return '\n\n'.join(rows_with_relevant_clusters.tolist()) if len(rows_with_relevant_clusters) else ''

@quiby
def get_indices_from_array(array, indices, indices_int_dtype=False, array_int_dtype=False):
    if indices_int_dtype:
        indices = indices.astype(int)
    if array_int_dtype:
        return array[indices].astype(int)
    return array[indices]

@quiby
def points_in_ellipse_indices(center, width, height, points):
    ellipse = ((center[0] - points[:,0])/width)**2 + ((center[1] - points[:,1])/height)**2 
    return np.where(ellipse<=1)[0]

## Prepare Figure and Axes

Create the figure and axes that will accomedate your interactive app. Usually this is an iterative process, tune the axes postions and sizes until you are pleased with the look and feel of your interactive app.

In [ ]:
cmap = plt.get_cmap('tab20')
num_chapters = len(chapters_abb)
chapter_colors = cmap(np.linspace(0, 1, num_chapters))

fig = plt.figure(0, figsize=(20,10))
fig.clf()
ax_scatter = fig.add_axes([0, 0, 0.5, 1])
ax_scatter.set_facecolor('xkcd:pale grey')
ax_scatter.grid(alpha=0.3)

ax_width = fig.add_axes([0.58, 0.0, 0.12, 0.05])
# ax_width.set_xlim(0, 10)

ax_height = fig.add_axes([0.58, 0.04, 0.12, 0.05])
# ax_height.set_xlim(0, 10)

ax_freq = fig.add_axes([0.58, 0.08, 0.12, 0.05])
# ax_freq.set_xlim(1,5000)

ax_bars = fig.add_axes([0.5, 0.35, 0.25, 0.65])
chapter_names = list(chapters_abb.values())
ax_bars.set_xticks(range(len(chapter_names)))
ax_bars.set_xticklabels(chapter_names, rotation=90)

ax_text = fig.add_axes([0.75, 0, 0.248, 1])
ax_text.set_yticks([]); # turn off axis labels

## Plot

**A general flow to plot in a "quibblerd" way:**

1. Define input quib parameters
2. Define any other parameter or transformation needed for the plot
3. Use one of matplotlib's supported plotting functions (use `is_quiby()` if you are not sure)
4. Add any widgets you want to allow full control of the input parameters

In [ ]:
# define variables to control scatter plot
frequency_threshold = iquib(100)
df_as_array = df.to_numpy()
indexes_to_plot = filter_by_frequency(df_as_array[:,2].astype(int), frequency_threshold)
above_threshold = get_indices_from_array(df_as_array, indexes_to_plot)
rows_to_plot = above_threshold[:, (6,7,5)] # two last columns are 2D TSNE of codes

# create scatter plot of TSNE
ax_scatter.scatter(rows_to_plot[:,0], rows_to_plot[:,1], s=8, 
                   c=get_indices_from_array(chapter_colors, rows_to_plot[:,2], indices_int_dtype=True))

# define ellipse quibbed variables
center = iquib(np.array([35, -28]))
width = iquib(12)
height = iquib(10)

# plot ellipse
t = np.linspace(0, 1, 60) * 2*np.pi
ax_scatter.plot(center[0] + width*np.cos(t), center[1] + height*np.sin(t), c='r', linewidth=2)
ax_scatter.plot(center[0], center[1], '+r', markersize=10, markeredgewidth=2)

# add dragging points to ellipse
t = np.linspace(1/8, 7/8, 4) * 2*np.pi
ax_scatter.plot(width*np.cos(t) + center[0], height*np.sin(t) + center[1], '*', 
                c='black', markersize=5, markeredgewidth=2)

# add slider for ellipse variables
slider_width = Slider(ax_width, 'Width', 0, 20, valinit=width)
slider_height = Slider(ax_height, 'Height', 0, 20, valinit=height)

# add slider for minimum frequency of ICD9 code in the dataset
Slider(ax_freq, valmin=1, valmax=5000, valinit=frequency_threshold, label='Min Frequency')

# calculate indcies of points inside ellipse
points_inside_ellipse = points_in_ellipse_indices(center, width, height, rows_to_plot)

# plot bars that represent the number of diseases inside the ellipse and add threshold to define cluster
chapter_to_plot = get_indices_from_array(above_threshold, (points_inside_ellipse, 5), array_int_dtype=True)
normalized_bins_height = q(np.bincount, chapter_to_plot, minlength=num_chapters)/(q(len, chapter_to_plot)+ 1e-6)

cluster_threshold = iquib(0.1)
ax_bars.bar(chapter_names, normalized_bins_height, color=chapter_colors)
ax_bars.axhline(cluster_threshold, linestyle='--', color='r')

# plot the descriptions of codes that does not pass the threshold
exception_codes = filter_text_by_frequency(normalized_bins_height, cluster_threshold, above_threshold[points_inside_ellipse])
ax_text.text(0.02, 0.95, exception_codes, fontsize=9, va='top', wrap=True);